# Laboratoire 2 : Arbre de désision, Bayes naïf et KNN
#### Département du génie logiciel et des technologies de l’information

| Étudiants             | Alexandre Bleau — BLEA14058906 / David Létinaud  — LETD05129708 / Thomas Lioret   — LIOT20069605|
|-----------------------|---------------------------------------------------------|
| Cours                 | GTI770 - Systèmes intelligents et apprentissage machine |
| Session               | Automne 2019                                            |
| Groupe                | 1                                                       |
| Numéro du laboratoire | 2                                                       |
| Professeur            | Prof. LOMBAERT                                          |
| Chargé de laboratoire | Pierre-Luc Delisle                                      |
| Date                  | 27/10/2019                                              |

## Introduction

Au cours de ce second laboratoire, nous allons étudier l’utilisation de trois algorithmes de classification. Nous utiliserons les arbres de décision comme vu précédemment, Bayes naïf et les K plus proches voisins (KNN). Nous allons aborder deux problèmes : la classification des galaxies ansi que la classification des courriels indésirables.
Afin de classer les galaxies en « spirales » ou « smooth », nous allons nous appuyer sur deux nouveaux algorithmes (Bayes naïf et KNN) ainsi que des prétraitements de données adaptées (MinMaxScaler). Dans le deuxième cas nous utiliserons également ces deux nouveaux algorithmes et ajouterons un cas de prétraitement supplémentaire (Discrétisation non-supervisée).
Enfin nous appliquerons le concept de validation croisée (cross-validation) et nous le comparerons à la validation holdout.

In [ ]:
dataset_path = "/home/ens/AQ38840/Desktop/data/data/csv/galaxy/galaxy_label_data_set.csv"
image_path = "/home/ens/AQ38840/Desktop/data/data/images/"
# Fichier de sortie
TP1_features_path = "/home/ens/AQ38840/Desktop/data/data/csv/galaxy/TP1_features.csv"

In [1]:
# Taille de rognage de l'image
crop_size = 180

TP1_feat_lignes = []

# Paramètres de chaque features determinees au TP1
fft_threshold = 140
color_center_size = 18
bp_calibration = [100,50]

def FeaturesProcess(img,cs_color,th_fft,nr_binaryPattern):
    """
    Fonction qui permet le calcul de chaque features d'img
    
    input :
        img (ndarray) : image quelconque
        cs_color (int) : taille du centre de l'image à prendre en compte pour calculer la moyenne du niveau de gris
        th_fft (int) : seuil à partir duquel on prend en compte les fréquences (strictement positif)
        nr_binaryPattern ([int,int]) : 
                    nr_binaryPattern[0] : nombre de points à prendre en compte sur le périmètre du cercle
                    nr_binaryPattern[1] : taille du rayon du cercle
    output : 
        (list) retourne la liste des features calculées
    
    """
    Features = []

    # Calculs des Features
    f_c = center_color(img,cs_color)
    f_fft = fourier_transform(img,th_fft)
    f_bp = binaryPatterns(img,nr_binaryPattern[0],nr_binaryPattern[1])  

    Features.append(f_c)   
    Features.append(f_fft)
    Features.append(f_bp)

    # Retourne les features calculés
    return Features    

########################################   Lecture   ########################################
# Lecture du fichier CSV
with open(dataset_path) as f:
    f_csv = csv.reader(f)
    next(f_csv) # On passe la 1ere ligne d'entête
    
    # Lecture ligne par ligne
    for ligne in f_csv:#,i in zip(f_csv,range(nb_img)):
        l_CSV = []
        # Lecture et rognage de l'image
        image = crop_center(io.imread( image_path + ligne[0] + ".jpg" ),crop_size,crop_size)
        X = FeaturesProcess(image, color_center_size, fft_threshold, bp_calibration)
        l_CSV.append(ligne[0])   # numéro d'image
        l_CSV.append(str(X[0]))  # feature 1
        l_CSV.append(str(X[1]))  # feature 2  
        l_CSV.append(str(X[2]))  # feature 3
        l_CSV.append(str(1 * (ligne[1]=="spiral"))) # classe de l'image
        TP1_feat_lignes.append(l_CSV)

f.close()
########################################   Ecriture   ########################################
print(TP1_feat_lignes)
with open(TP1_features_path, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(TP1_feat_lignes)
    writeFile.close()

SyntaxError: invalid syntax (<ipython-input-1-83fe594449e9>, line 1)

## Question 1
### Détails des ensembles produits
L'ensemble de données ainsi produit contient toutes les informations utiles pour de l'apprentissage supervisée et sont disposées de cette façon :
['numero_image', 'feature_1','feature_2','feature_3', 'classe_image']

Le fichier TP1_features.csv sera ainsi utilisé pour compléter les features contenu dans le fichiers 'galaxy_feature_vectors.csv'.

## Question 2
### Approche de validation proposée et justification
Nous allons ici utiliser 3 approches de classification différentes (arbre de décision, Bayes naïf et KNN) pour traiter des mails et des images.

Pour valider et comparer l'efficacité des modèle nous utiliserons la précision en combinaison avec le F1-score.

La précision mesure quelle proportion de bonne classification. Elle permet de savoir immédiatement si un modèle est correctement entraîné et comment il peut fonctionner en général. Toutefois, il ne donne pas d'informations détaillées concernant son application au problème, c'est pour cela qu'on le combine avec le F1 score.

F1 score est une mesure globale de la précision d'un modèle de classification binaire, qui combine précision et rappel. La précision répond à la question : Quelle proportion d'identifications positives était effectivement correcte ? alors que le Rappel (ou sensitivité) : Quelle proportion de résultats positifs réels a été identifiée correctement ?
Un bon F1 score signifie que le modèle prédit peu de faux positifs et peu de faux négatifs. Le F1 score est considéré comme parfait lorsqu'il est de 1, alors que le modèle est un échec total lorsqu'il est de 0.

Nous avons utilisé la méthode de validation holdout (split stratify) qui est la plus simple et la plus rapide pour tester les algorithmes de classification. Stratify nous permet de garder la même répartition des données dans les données de tests et d'entrainement.

## Question 3
### Matrice des expérimentations

## Question 3
### Étude des hyperparamètres et des modèles
Pour choisir les meilleurs hyperparamètres des différents modèles nous allons mesurer les performances de prédiction chaque valeur des hyperparamètres.

Pour ce faire, nous allons entraîner chaque modèle avec 100% du dataset. La méthode de validation holdout nous permet de le faire. Chaque modèle est entrainé successivement avec des valeurs d'hyperparamètres compris dans un intervalle choisi judicieusement. La précision et le F1-score sont à chaque fois sauvegardés dans une matrice. On pourra ainsi choisir les hyperparamètres qui donne les meilleures performances au modèle.

## Question 4
### Impact de la taille des ensembles de données sur la performance de classification
Plus l'ensemble de données est grand et plus le modèle est efficace. Si on utilise un trop petit nombre d'échantillons pour entraîner un modèle, il risque de ne pas être très performant lors de la prédiction. En effet, il est probable que le nombre d'échantillons atypiques soient sur-représentés.
Toutefois, de part le fonctionnement de la cross-validation, cette méthode s'avère plus efficace quand il s'agit de traiter des petits ensembles de données.

## Question 5
### Impact du bruit dans les ensembles de données sur la performance de classification
Dans un ensemble de données, le bruit peu diminuer les performances d'un modèle. Il est important de prendre un nombre d'échantillons assez grand pour éviter d'entraîner un modèle avec un nombre d'échantillons atypiques trop grand par rapport aux autres.
Toutefois, le bruit peu avoir des impacts différents sur certains modèles. On peut prendre comme exemple KNN. Plus le bruit est important plus il est préférable d'augmenter l'hyperparamètre K (nombre de voisins) pour lisser l'effet du bruit.

## Question 6
### Discussion sur la nature des données


## Question 7
### Recommandations

## Question 8
### Améliorations possibles
Dans ce lab, nous avons exploré 3 types de modèles, mais un autre modèle pourrait peut-être être plus performant dans notre cas. On pourrait tester un plus grand nombre de modèles comme par exemple un réseau de neurones ou des SVM. 

## Conclusion
Ici, le problème de base était la classification de mail et d'images. Toutefois, pour obtenir les meilleurs résultats, cela soulève plusieurs problématiques. Il a été par exemple utile de tester plusieurs types de modèles pour être capable de choisir celui qui performe le mieux. D'autre part, l'analyse de performances a également été cruciale. Il a été nécessaire d'étudier les hyperparamètres qui donnent les meilleurs résultats.